# Main model run



### Basic setups

In [2]:
# !pip install transformers
# !pip install spacy
# !python -m spacy download en_core_web_md
# !pip install annoy
# !pip install sentence_transformers
# !pip install evaluate
# !pip install bert_score

In [3]:
# vizualization library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

# pytorch library
import torch # the main pytorch library
import torch.nn.functional as f # the sub-library containing different functions for manipulating with tensors

# huggingface's transformers library
from transformers import BertModel, BertTokenizer, BertForSequenceClassification

from annoy import AnnoyIndex

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import pandas as pd
import zipfile
from urllib.request import urlretrieve

from nltk.tokenize import word_tokenize
import string
import tqdm
import pickle
import gc
from evaluate import load

In [4]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import string

import nltk
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
from transformers import T5ForConditionalGeneration,T5Tokenizer
import sentencepiece

In [6]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from transformers import  AdamW

#to avoid warnings
import warnings
warnings.filterwarnings('ignore')


In [7]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)


### Download data


In [9]:
# Loading the zip file and extracting a zip object
with zipfile.ZipFile("main_model_train.zip", 'r') as zip_file:
    zip_file.extract("main_model_train/main_model_train.csv")

with zipfile.ZipFile("word_embeddings.zip", 'r') as zip_file:
    zip_file.extract("word_embeddings/word_embeddings.pkl")

In [55]:
# Read files
train_df = pd.read_csv("main_model_train/main_model_train.csv", index_col=0)
eval_df = pd.read_csv("main_model_eval.csv", index_col=0)
test_df = pd.read_csv("main_model_test.csv", index_col=0)

train_df

,input_text,target_text
0,"I knew mushrooms, and I don't give a shit. """,I didn't know and I don't much care.”
1,Keep on lying like that. You're liable to get ...,you can keep lying like that and you'll be abl...
2,I was probably worth killing then.,I probably stood up for the assassination then.
3,It's bad enough we scared the shit out of them...,we already scared them off with the T-shirt wi...
4,"wake up, you corpses!",Heave and wake the dead!
...,...,...
462216,"Look, suck it, dude.","look, bite it, man."
462217,And if I have to look at that stupid picture o...,and if I have to take another look at the phot...
462218,I miss working with her because she's so godda...,I miss working with her because she does damn ...
462219,"Wow, this place is fuckin' incredible.","wow, this is incredible."


In [65]:
# Load words embeddings
with open('word_embeddings/word_embeddings.pkl', "rb") as fIn:
    word_embeddings = pickle.load(fIn)

words = word_embeddings['word']
embeddings = word_embeddings['embedding']

# Free memory
word_embeddings = None
gc.collect()

0

## Synonyms finding

By embedding of the words, get_synonym will return top k best synonyms (by embeddings cosine similarity). Synonyms are finded by AnnoyIndex that is trained on most frequent words embeddings

In [16]:
def preprocess(sent):
    """
    Preprocess name of the dataset point
    Lowercased without punctuation and stop word
    Return list of preprocessed words from the sent
    """
    res = []

    try:
        words = word_tokenize(sent)
    except:
        print(f"\nTokenization fails for {sent}")
        return []

    for word in words:
        # Delete punctuation
        sent = sent.translate(str.maketrans("", "", string.punctuation))
        # Split by a free space
        word = word.strip()
        # Lowercase text
        word = word.lower()

        # Ignore free space
        if len(word) > 0:
            res.append(word)

    # Return list of preprocessed words from the sent
    return res

In [17]:
model_similarity = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')  # multi-language model

def embed(text):
    global model_similarity
    return model_similarity.encode([text], convert_to_tensor=False)[0]


In [72]:
def get_vector_index(embeddings, start=0):
    # Initialize index
    annoy3 = AnnoyIndex(embeddings.shape[1], 'angular')
    i = start
    for embedding in tqdm.tqdm(embeddings):
        try:
            # Add non-zero embeddings in the index
            # Because points with zero embedding
            # is given to "unknown" words and phrases
            # embedding = embed(embedding)
            if np.sum(np.abs(embedding)) != 0:
                annoy3.add_item(i, embedding)
        except:
            pass
        i += 1

    # Build 37 trees
    annoy3.build(37)
    print("Index is constructed")
    annoy3.save('annoy_index.ann')

    # Return resulting index
    return annoy3


def get_kNN_embeddings(embedding, k, index):
    # Obtain nearest neighbours
    return index.get_nns_by_vector(embedding, k)

In [47]:
# embeddings_i = np.zeros((embed("hello").shape[0], 1000))
# embeddings.shape

(333331, 384)

In [75]:
index = AnnoyIndex(embed("hello").shape[0], 'angular')
index.load('annoy_index.ann')
# index = get_vector_index(embeddings[:100000])

100%|██████████| 100000/100000 [00:03<00:00, 28188.78it/s]


Index is constructed


In [102]:
def get_synonyms(word, k=100):
    global index
    ids = get_kNN_embeddings(embed(word), k, index)
    return words[ids]

get_synonyms("person", k=10)

array(['individuals', 'socio', 'vrouw', 'savant', 'kas', 'humans',
       'personalise', 'temasek', 'une', 'utters'], dtype=object)

## Paraphraser

In [92]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer

paraphraser_bart = BartForConditionalGeneration.from_pretrained('eugenesiow/bart-paraphrase')
paraphraser_bart = paraphraser_bart.to(device)
tokenizer_paraphr_bart = BartTokenizer.from_pretrained('eugenesiow/bart-paraphrase')

In [62]:
gc.collect()

96

In [93]:
def paraphrase_sent(input_sentence):
    global tokenizer_paraphr_bart, paraphraser_bart, device
    batch = tokenizer_paraphr_bart(input_sentence, return_tensors='pt').to(device)
    generated_ids = paraphraser_bart.generate(batch['input_ids'])
    generated_sentence = tokenizer_paraphr_bart.batch_decode(generated_ids, skip_special_tokens=True)
    return generated_sentence[0]

In [94]:
paraphrase_sent("I will fucking hate you, stupid nigger")

'I will hate you, stupid nigger.'

## Sentence Similarity

In [26]:
model_similar = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

In [27]:
def sentence_similarity(sent1, sent2):
    global model_similar
    sentences = [
        sent1,
        sent2
    ]
    embedding = model_similar.encode(sentences, convert_to_tensor=False)

    cosine_scores = util.cos_sim(embedding, embedding)

    return cosine_scores[0][1].item()


# Example
sentence_similarity("I'll fucking hate you", "I'll really hate you")

0.9447494745254517

## Toxicity score

In [41]:
from transformers import pipeline

toxic_classificator = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')

def toxicity(text):
    # Returns toxicity of the text
    global toxic_classificator
    hypothesis_template = 'This text is {}.'
    prediction = toxic_classificator(text, ["toxic"], hypothesis_template=hypothesis_template, multi_label=True)
    return prediction['scores'][0]

# Example
print(toxicity("hello, dear"))
print(toxicity("fucking"))

0.0005157999694347382
0.9027175307273865


## Predict

In [103]:
def process_sentence(sent, max_synonyms=3):
    words = preprocess(sent)
    result = []
    for ind, word in enumerate(words):
        # Special cases after tokenizer
        if word in stop_words or word in string.punctuation or toxicity(word) <= 0.5:
            result.append(word)
            continue

        best_synonym = ""
        best_simil_score = sentence_similarity(sent, " ".join(words[:ind]+words[ind+1:]))
        # May be add analyzing skipping !!!
        counter = 0
        for synonym in get_synonyms(word):
            toxic = toxicity(synonym)
            if synonym in stop_words or toxic <= 0.5:
                # Create a sentence as initial
                potential_sentence = words.copy()
                # And replace toxic word with a synonym
                potential_sentence[ind] = synonym
                potential_similarity = sentence_similarity(sent, " ".join(potential_sentence))

                # Better synonym in the context
                if potential_similarity > best_simil_score:
                    best_simil_score = potential_similarity
                    best_synonym = synonym
                    # print(f"Better synonym for {word} is {synonym}")

                # Non-toxic word was analyzed
                counter += 1

            if counter >= max_synonyms:
                # Analyze only top max_synonyms
                break
        result.append(best_synonym)
    # print("Before paraphrasing:", " ".join(result))
    # Paraphrase
    return paraphrase_sent(" ".join(result))



### Visualization

In [30]:
bertscore = load("bertscore")

In [96]:
arr_s = ["oh shut up, goddamn", "I'll fucking hate you, stupid idiot" ]

for i in arr_s:
    print("Initial text:", i)
    res = process_sentence(i)
    print("Result:", res)
    score = bertscore.compute(predictions=[res], references=[i], lang="en")
    print("Bart score is ", score)

    print()

Initial text: oh shut up, goddamn
Before paraphrasing: oh shute up , auck
Result: oh Shute Up, Auck
Bart score is  {'precision': [0.8297189474105835], 'recall': [0.8473272919654846], 'f1': [0.838430643081665], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.35.0)'}

Initial text: I'll fucking hate you, stupid idiot
Before paraphrasing: i wird  odio you , dumb idiots
Result: i wird  odio you, dumb idiots.
Bart score is  {'precision': [0.8560117483139038], 'recall': [0.8811811208724976], 'f1': [0.8684141039848328], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.35.0)'}



In [99]:
final_score = 0
for ind, row in test_df[:5].iterrows():
    print("input text: ", row['input_text'])

    pred = process_sentence(row['input_text'])
    print("Predictions: ", pred)
    print("Target text: ", row['target_text'])
    print()

    score = bertscore.compute(predictions=[pred], references=[row['target_text']], lang="en")
    print("BERT F1 Score:", score['f1'])
    print("Toxicity:", toxicity(pred))
    print("\n")

input text:  He saved our ass in Hereford, probably my wife and daughter, too."
Target text:  and he probably saved my wife and my daughter. "
Before paraphrasing: he saved our assuring in hereford , probably my wife and daughter , too . ''
Predictions:  He saved our assuring in hereford, probably my wife and daughter too.
BERT F1 Score: [0.8928162455558777]

input text:  it's like the fucking birds on the trees sang her name.
Target text:  It's like the friggin' birds are singing her name in the trees.
Before paraphrasing: it  like the  birds on the trees sang her name .
Predictions:  Like the birds on the trees sang her name.
BERT F1 Score: [0.9190836548805237]

input text:  Stand away from that lady!.
Target text:  leave the lady alone!
Before paraphrasing: stand cuti from that lady ! .
Predictions:  stand cuti from that lady!
BERT F1 Score: [0.8604728579521179]

input text:  "All right, but I'll have to use it later when I gag you."
Target text:  "all right, but I have to use it as

In [109]:
final_score = 0
for ind, row in test_df[:4].iterrows():
    print("input text: ", row['input_text'])
    print("Target text: ", row['target_text'])

    pred = process_sentence(row['input_text'])
    print("Predictions: ", pred)
    print()

    score = bertscore.compute(predictions=[pred], references=[row['target_text']], lang="en")
    print("BERT F1 Score: %.3f" % score['f1'][0])
    print("Toxicity: %.3f" % toxicity(pred))
    print("\n")

input text:  He saved our ass in Hereford, probably my wife and daughter, too."
Target text:  and he probably saved my wife and my daughter. "
Predictions:  He saved our assuring in hereford, probably my wife and daughter too.

BERT F1 Score: 0.893
Toxicity: 0.001


input text:  it's like the fucking birds on the trees sang her name.
Target text:  It's like the friggin' birds are singing her name in the trees.
Predictions:  Like the birds on the trees sang her name.

BERT F1 Score: 0.919
Toxicity: 0.001


input text:  Stand away from that lady!.
Target text:  leave the lady alone!
Predictions:  stand cuti from that lady!

BERT F1 Score: 0.860
Toxicity: 0.756


input text:  "All right, but I'll have to use it later when I gag you."
Target text:  "all right, but I have to use it as a gag later."
Predictions:  `` All right, but I wird have to use it later when I gaggle

BERT F1 Score: 0.906
Toxicity: 0.001




### Evaluation

In [110]:
final_score = [0, 0]
length_evaluation = 1000
for ind, row in test_df[:length_evaluation].iterrows():
    # print("input text: ", row['input_text'])

    pred = process_sentence(row['input_text'])
    # print("Predictions: ", pred)
    # print("Target text: ", row['target_text'])
    # print()

    score = bertscore.compute(predictions=[pred], references=[row['target_text']], lang="en")
    final_score[0] += score['f1'][0]
    final_score[1] += toxicity(pred)
    # print("Toxicity:", toxicity(pred))
    # print("\n")
final_score[0] /= length_evaluation
final_score[1] /= length_evaluation
print("Evaluation results:")
print("BERT F1 score: %.3f" % final_score[0])
print("Toxicity: %.3f" % final_score[1])

Evaluation results:
BERT F1 score: 0.891
Toxicity: 0.252


In [119]:
# Count number of badly predicted as toxic
toxic_score = [0, 0]
length_evaluation = 1000

for ind, row in test_df[:length_evaluation].iterrows():

    pred = process_sentence(row['input_text'])

    # Compute toxicity level
    if toxicity(pred) < 0.5:
        toxic_score[0] += 1
    else:
        toxic_score[1] += 1


print("Probability distribution over simulation:")
print("\nProbability of getting the non-toxic sentence: %.3f" % (toxic_score[0] / sum(toxic_score)), "and toxic %.3f" % (toxic_score[1] / sum(toxic_score)))


Probability distribution over simulation:

Probability of getting the non-toxic sentence: 0.752 and toxic 0.248
